In [26]:
import angr
import os
import argparse
import monkeyhex
from angrutils import *
import subprocess
import graphviz

In [2]:
proj = angr.Project("./test", load_options={'auto_load_libs': False})
print(proj)
#load the binary into the project

WARNING | 2022-11-18 15:23:59,707 | cle.loader | The main binary is a position-independent executable. It is being loaded with a base address of 0x400000.


<Project ./test>


In [3]:
proj.arch #This will give me the archatecture of the device the binary was created on.

<Arch AMD64 (LE)>

In [4]:
proj.entry #This will give me the binary entry point the right starting hex

0x4010a0

In [5]:
proj.filename #This will give the location of the binary

'./test'

In [6]:
proj.loader

<Loaded test, maps [0x400000:0x807fff]>

In [7]:
proj.loader.shared_objects

OrderedDict([('test', <ELF Object test, maps [0x400000:0x404017]>),
             ('extern-address space',
              <ExternObject Object cle##externs, maps [0x600000:0x607fff]>),
             ('cle##tls',
              <ELFTLSObjectV2 Object cle##tls, maps [0x700000:0x71500f]>)])

In [8]:
proj.loader.min_addr

0x400000

In [9]:
proj.loader.max_addr

0x807fff

In [10]:
proj.loader.main_object

<ELF Object test, maps [0x400000:0x404017]>

In [11]:
proj.loader.main_object.execstack

False

In [12]:
proj.loader.main_object.pic

True

In [13]:
cfg = proj.analyses.CFGFast(data_references=True, normalize=True)
print(cfg.graph)
#create the cfg using CFGFast function from anger to get a fast a complete picture of all the blocks

DiGraph with 67 nodes and 78 edges


In [14]:
nodelist = list(cfg.graph.nodes())
print(nodelist)
#Make a list to hold all of the nodes detected

numofnodes = len(nodelist)
print(numofnodes)
#print the total num of nodes

[<CFGNode _start [46]>, <CFGNode _init [20]>, <CFGNode deregister_tm_clones [19]>, <CFGNode register_tm_clones [36]>, <CFGNode __do_global_dtors_aux [13]>, <CFGNode frame_dummy [9]>, <CFGNode main [58]>, <CFGNode __libc_csu_init [49]>, <CFGNode __libc_csu_fini [5]>, <CFGNode _fini [13]>, <CFGNode __libc_start_main [0]>, <CFGNode _init+0x16 [5]>, <CFGNode _init+0x14 [2]>, <CFGNode deregister_tm_clones+0x28 [1]>, <CFGNode deregister_tm_clones+0x13 [12]>, <CFGNode register_tm_clones+0x38 [1]>, <CFGNode register_tm_clones+0x24 [12]>, <CFGNode __do_global_dtors_aux+0xd [14]>, <CFGNode __do_global_dtors_aux+0x38 [1]>, <CFGNode 0x401090[11]>, <CFGNode deregister_tm_clones+0x1f [2]>, <CFGNode register_tm_clones+0x30 [2]>, <CFGNode __do_global_dtors_aux+0x27 [5]>, <CFGNode __do_global_dtors_aux+0x1b [12]>, <CFGNode scanf [0]>, <CFGNode  [11]>, <CFGNode UnresolvableJumpTarget [0]>, <CFGNode __do_global_dtors_aux+0x2c [9]>, <CFGNode __libc_csu_init+0x31 [6]>, <CFGNode __libc_csu_init+0x56 [15]>, 

In [15]:
edgelist = list(cfg.graph.edges())
print(edgelist)
#Make a list to hold all of the edges detected

numofedges = len(edgelist)
print(numofedges)
#print the total num of edges

[(<CFGNode _start [46]>, <CFGNode __libc_start_main [0]>), (<CFGNode _init [20]>, <CFGNode _init+0x16 [5]>), (<CFGNode _init [20]>, <CFGNode _init+0x14 [2]>), (<CFGNode deregister_tm_clones [19]>, <CFGNode deregister_tm_clones+0x28 [1]>), (<CFGNode deregister_tm_clones [19]>, <CFGNode deregister_tm_clones+0x13 [12]>), (<CFGNode register_tm_clones [36]>, <CFGNode register_tm_clones+0x38 [1]>), (<CFGNode register_tm_clones [36]>, <CFGNode register_tm_clones+0x24 [12]>), (<CFGNode __do_global_dtors_aux [13]>, <CFGNode __do_global_dtors_aux+0xd [14]>), (<CFGNode __do_global_dtors_aux [13]>, <CFGNode __do_global_dtors_aux+0x38 [1]>), (<CFGNode frame_dummy [9]>, <CFGNode register_tm_clones [36]>), (<CFGNode main [58]>, <CFGNode 0x401090[11]>), (<CFGNode main [58]>, <CFGNode main+0x3a [10]>), (<CFGNode __libc_csu_init [49]>, <CFGNode _init [20]>), (<CFGNode __libc_csu_init [49]>, <CFGNode __libc_csu_init+0x31 [6]>), (<CFGNode __libc_start_main [0]>, <CFGNode __libc_csu_init [49]>), (<CFGNode 

In [16]:
nodelist1 = [node for node in list(cfg.graph.nodes()) if node.block !=None]
#This is a list iterator meant to remove all nodes with empty node blocks

print(nodelist1)
print(len(nodelist1))
#Print this new node list and its length

[<CFGNode _start [46]>, <CFGNode _init [20]>, <CFGNode deregister_tm_clones [19]>, <CFGNode register_tm_clones [36]>, <CFGNode __do_global_dtors_aux [13]>, <CFGNode frame_dummy [9]>, <CFGNode main [58]>, <CFGNode __libc_csu_init [49]>, <CFGNode __libc_csu_fini [5]>, <CFGNode _fini [13]>, <CFGNode _init+0x16 [5]>, <CFGNode _init+0x14 [2]>, <CFGNode deregister_tm_clones+0x28 [1]>, <CFGNode deregister_tm_clones+0x13 [12]>, <CFGNode register_tm_clones+0x38 [1]>, <CFGNode register_tm_clones+0x24 [12]>, <CFGNode __do_global_dtors_aux+0xd [14]>, <CFGNode __do_global_dtors_aux+0x38 [1]>, <CFGNode 0x401090[11]>, <CFGNode deregister_tm_clones+0x1f [2]>, <CFGNode register_tm_clones+0x30 [2]>, <CFGNode __do_global_dtors_aux+0x27 [5]>, <CFGNode __do_global_dtors_aux+0x1b [12]>, <CFGNode  [11]>, <CFGNode __do_global_dtors_aux+0x2c [9]>, <CFGNode __libc_csu_init+0x31 [6]>, <CFGNode __libc_csu_init+0x56 [15]>, <CFGNode main+0x3a [10]>, <CFGNode main+0x44 [12]>, <CFGNode main+0x52 [12]>, <CFGNode  [11]

In [17]:
print("number of nodes in the graph:", numofnodes)
print("number of edges in the graph:", numofedges)

number of nodes in the graph: 67
number of edges in the graph: 78


In [18]:
main = proj.loader.main_object.get_symbol("main")
print(main)

<Symbol "main" in test at 0x401189>


In [19]:
start_state = proj.factory.blank_state(addr=main.rebased_addr)
print(start_state)
 

<SimState @ 0x401189>


In [20]:
plot_cfg(cfg, "binary_test_cfg", format="png", asminst=True, remove_imports=True, remove_path_terminator=True)  

In [21]:
plot_cfg(cfg, "binary_test_cfg", format="dot", asminst=True, remove_imports=True, remove_path_terminator=True)  

In [22]:
allIns = set()

In [23]:
for node in nodelist1:
    print(node)
    for ins in ((node.block.disassembly.insns)):
        print(ins.mnemonic)
        allIns.add(ins.mnemonic)
        


<CFGNode _start [46]>
endbr64
xor
mov
pop
mov
and
push
push
lea
lea
lea
call
<CFGNode _init [20]>
endbr64
sub
mov
test
je
<CFGNode deregister_tm_clones [19]>
lea
lea
cmp
je
<CFGNode register_tm_clones [36]>
lea
lea
sub
mov
shr
sar
add
sar
je
<CFGNode __do_global_dtors_aux [13]>
endbr64
cmp
jne
<CFGNode frame_dummy [9]>
endbr64
jmp
<CFGNode main [58]>
endbr64
push
mov
sub
mov
mov
mov
mov
xor
lea
mov
lea
mov
call
<CFGNode __libc_csu_init [49]>
endbr64
push
lea
push
mov
push
mov
push
mov
push
lea
push
sub
sub
call
<CFGNode __libc_csu_fini [5]>
endbr64
ret
<CFGNode _fini [13]>
endbr64
sub
add
ret
<CFGNode _init+0x16 [5]>
add
ret
<CFGNode _init+0x14 [2]>
call
<CFGNode deregister_tm_clones+0x28 [1]>
ret
<CFGNode deregister_tm_clones+0x13 [12]>
mov
test
je
<CFGNode register_tm_clones+0x38 [1]>
ret
<CFGNode register_tm_clones+0x24 [12]>
mov
test
je
<CFGNode __do_global_dtors_aux+0xd [14]>
push
cmp
mov
je
<CFGNode __do_global_dtors_aux+0x38 [1]>
ret
<CFGNode 0x401090[11]>
endbr64
bnd jmp
<CFGNo

In [24]:
print(allIns)

{'test', 'jmp', 'and', 'cmp', 'sub', 'hlt', 'sar', 'ret', 'pop', 'bnd jmp', 'xor', 'call', 'je', 'shr', 'push', 'add', 'leave', 'jne', 'mov', 'lea', 'nop', 'endbr64'}


In [25]:
print("number of different instruction types:", len(allIns))

number of different instruction types: 22
